In [1]:
# add quantkit to path
import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))

In [2]:
import numpy as np
import pandas as pd

import quantkit.bt as bt
import quantkit.bt.frequency as frequency
import quantkit.bt.signals as signals
import quantkit.bt.portfolio_management as portfolio_management
import quantkit.bt.risk_management as risk_management
import quantkit.bt.weighting_schemes as weighting_schemes
import quantkit.bt.technical_analysis as technical_analysis
import quantkit.bt.util as util

import warnings
warnings.filterwarnings('ignore')

In [3]:
names = ["foo", "bar", "rf"]
dates = pd.date_range(
    start="2017-01-01", end="2017-12-31", freq=pd.tseries.offsets.BDay()
)
n = len(dates)
rdf = pd.DataFrame(np.zeros((n, len(names))), index=dates, columns=names)

np.random.seed(1)
rdf["foo"] = np.random.normal(loc=0.1 / n, scale=0.2 / np.sqrt(n), size=n)
rdf["bar"] = np.random.normal(loc=0.04 / n, scale=0.05 / np.sqrt(n), size=n)
rdf["rf"] = np.random.normal(loc=0.02 / n, scale=0.01 / np.sqrt(n), size=n)

pdf = 100 * np.cumprod(1 + rdf)

In [4]:
children = [ bt.Security( name ) for name in pdf.columns ]

# algo to fire on the beginning of every month and to run on the first date
runMonthlyAlgo = frequency.RunMonthly(
    run_on_first_date=True
)


select_algo = signals.SelectAll()

# algo to set the weights in the temp dictionary\
weights = pd.Series([0.6, 0.4, 0.0], index=rdf.columns)
weighSpecifiedAlgo = weighting_schemes.SpecifiedWeight(**weights)

# algo to rebalance the current weights to weights set in temp dictionary
rebalAlgo = portfolio_management.Rebalance()

# a strategy that rebalances monthly to specified weights
s = "monthly"
strat = bt.Strategy(
    s, 
    algos = [
        runMonthlyAlgo,
        select_algo, 
        weighSpecifiedAlgo, 
        rebalAlgo
    ],
    children=children
)

# set integer_positions=False when positions are not required to be integers(round numbers)
backtest = bt.Backtest(strat, pdf, integer_positions=False)

res = bt.run(backtest)

# set riskfree as the rf index
res.set_riskfree_rate(pdf["rf"])

In [5]:
res.get_data()

,price,value,notional_value,cash,fees,flows
2017-01-01,100.000000,1.000000e+06,0.000000e+00,1.000000e+06,0.0,1000000.0
2017-01-02,100.000000,1.000000e+06,0.000000e+00,0.000000e+00,0.0,0.0
2017-01-03,99.384719,9.938472e+05,9.938472e+05,0.000000e+00,0.0,0.0
2017-01-04,99.121677,9.912168e+05,9.912168e+05,0.000000e+00,0.0,0.0
2017-01-05,98.316364,9.831636e+05,9.831636e+05,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...
2017-12-25,120.459991,1.204600e+06,1.204600e+06,-1.218723e-10,0.0,0.0
2017-12-26,122.182438,1.221824e+06,1.221824e+06,-1.218723e-10,0.0,0.0
2017-12-27,122.833522,1.228335e+06,1.228335e+06,-1.218723e-10,0.0,0.0
2017-12-28,123.289067,1.232891e+06,1.232891e+06,-1.218723e-10,0.0,0.0


In [6]:
res.get_weights()

,monthly,monthly>foo,monthly>bar,monthly>rf
2017-01-01,1.0,0.000000,0.000000,0.0
2017-01-02,1.0,0.000000,0.000000,0.0
2017-01-03,1.0,0.599366,0.400634,0.0
2017-01-04,1.0,0.597251,0.402749,0.0
2017-01-05,1.0,0.594361,0.405639,0.0
...,...,...,...,...
2017-12-25,1.0,0.590946,0.409054,0.0
2017-12-26,1.0,0.597662,0.402338,0.0
2017-12-27,1.0,0.599041,0.400959,0.0
2017-12-28,1.0,0.600237,0.399763,0.0


In [7]:
res.get_positions()

,foo,bar,rf
2017-01-01,0.000000,0.000000,0.0
2017-01-02,5879.285683,3998.068018,0.0
2017-01-03,5879.285683,3998.068018,0.0
2017-01-04,5879.285683,3998.068018,0.0
2017-01-05,5879.285683,3998.068018,0.0
...,...,...,...
2017-12-25,5324.589093,4673.239436,0.0
2017-12-26,5324.589093,4673.239436,0.0
2017-12-27,5324.589093,4673.239436,0.0
2017-12-28,5324.589093,4673.239436,0.0
